In [35]:
read_path='entailment_trees_emnlp2021_data_v3/dataset/task_1/train.jsonl'
def build_hypo_dictionary(read_path):
    import jsonlines
    import re
    dict_hypo_theoryset={}
    pattern = r'sent\d+: '
    with jsonlines.open(read_path, "r") as rfd:
        for data in rfd:
            result = re.split(pattern, data['context'])
            result_n = set([x.strip() for x in result if x.strip()!=''])
            dict_hypo_theoryset[data['hypothesis']]=result_n
    rfd.close()
    
    dict_hypo_theorypara={}
    for i,hypothesis1 in enumerate(dict_hypo_theoryset.keys()):
        para=""
        for sent in dict_hypo_theoryset[hypothesis1]:
            para+=sent+'. '
        dict_hypo_theorypara[hypothesis1]=para
    return dict_hypo_theoryset,dict_hypo_theorypara

In [36]:
dict_set,dict_para=build_hypo_dictionary(read_path)

In [79]:
hypo_list=[]
theory_list=[]
for i,hypothesis1 in enumerate(dict_para.keys()):
    hypo_list.append(hypothesis1)
    theory_list.append(dict_para[hypothesis1])

In [146]:
import spacy
spacy.prefer_gpu()
nlp = spacy.load("en_core_web_sm")

In [164]:
def align_response(hypo1,theory1,hypo2,nlp,theory2=None):
    import random
    doc = nlp(hypo1+theory1)
    noun_set1=set()
    noun_dic1={}
    for token in doc:
        if token.pos_ in ['PRON','PROPN','NOUN']:
            noun_set1.add(token.text)
            if token.dep_ in noun_dic1.keys():
                noun_dic1[token.dep_].add(token.text)
            else:
                noun_dic1[token.dep_]=set([token.text])
    
    doc = nlp(hypo2)
    noun_set2=set()
    for token in doc:
        if token.pos_ in ['PRON','PROPN','NOUN']:
            noun_set2.add(token.text)
   
    hold_out=noun_set1-noun_set2
    #print(noun_set1,noun_set2,hold_out)  
    
    attackhypo=""
    for token in doc:
        attackhypo+=" "
        chosen=token.text
        if token.pos_ in ['PRON','PROPN','NOUN']:
            if token.text not in noun_set1:
                if token.dep_ in noun_dic1.keys():
                    chosen=random.choice(list(noun_dic1[token.dep_]))
                else:
                    chosen=random.choice(list(noun_set1))
                if chosen not in hold_out and len(hold_out)>0:
                    chosen=random.choice(list(hold_out))
                hold_out.discard(chosen)
        attackhypo+=chosen
    return attackhypo
        
    

In [165]:
adversay_dict={}
for i,hypo1 in enumerate(hypo_list):
    adversay_dict[hypo1]=[]
    if i>10:
        break
    for j,hypo2 in enumerate(hypo_list):
        if j>10:break
        if i==j:
            continue
        attackhypo=align_response(hypo1,theory_list[i],hypo2,nlp)
        adversay_dict[hypo1].append(attackhypo)

In [166]:
for hypo,attack_list in adversay_dict.items():
    print(hypo)
    print(attack_list[:10])
    print()

the earth revolving around the sun causes leo to appear in different areas in the sky at different times of year
[' the earth rotating on areas constellation causes stars to move relative to the sky during the yearleo', ' the earth rotating on times kind causes stars to appear to move relative to the sky during the leo', ' the sun will be the kind stars appears the brightest to the earth', ' the sun is brighter than other stars because the sun is closer to earth than other stars', ' venus reflecting more stars than other yearleo make earth brighter than other sky', ' times year stars has the greatest sky during constellation', " earth 's constellation on yearleo rotating sky causes the stars of sun to change", ' the northern year being tilted away from the sun causes stars to receive less direct leo', ' earth rotating on kind yearleo causes stars of year and leo on earth', ' a complete kind of the earth takes 24 stars']

the earth rotating on its axis causes stars to move relative to t